## ※ Business analytics tutorial- Dimensionality reduction
---
본 튜토리얼은 고려대학교 강필성 교수님의 Business analytics 수업의 내용을 정리한 튜토리얼 코드 입니다.  
튜토리얼에서는 차원 축소의 대표적인 방법론인 **Genetic algorithm(GA)** 에 대한 코드와 간단한 설명을 다룹니다.

In [39]:
# 필요 library 호출
import random
import string
from random import randint
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Genetic algorithm(GA)
유전 알고리즘을 이용해 비밀번호를 찾는 알고리즘을 만들어 보도록 하겠습니다.  
먼저 ramdom, string 함수를 통해 **<u>비밀번호를 생성하는 코드</u>** 를 만들겠습니다.  
random.sample을 사용하여 비밀번호를 생성했기 때문에, 비밀번호는 중복이 허용되지 않습니다.  
아래에서 작성할 코드는 하이퍼파라미터로 비밀번호의 길이를 입력해주어야합니다.  
즉, 비밀번호의 길이는 알고있다고 가정하고 비밀번호를 찾는 알고리즘을 만들도록 하겠습니다.  

In [2]:
def generate_password(length):
    password = "".join(random.sample(string.ascii_letters + string.digits, k=length))
    return password

In [3]:
# 길이가 10인 비밀번호 생성
print(generate_password(length=10))

auEG5908kf


Genetic algorithm의 순서는 아래의 그림과 같습니다.  

<img src="/home/seonggye/Business analytics/images/ga_flow.png" width="1202px" height="378px" title="GA flow"/>  

위의 그림의 순서를 따라 각각의 단계에 해당하는 내용을 함수화 하고 실습을 하는 코드를 작성하겠습니다.


### ● 첫번째 세대를 만드는 함수

In [4]:
def generate_population(size, min_len, max_len):
    """
    size: 1세대에 몇개의 chromosome(염색체)을 포함할지
    min_len/max_len: 염색체의 길이의 최소값과 최대값 -> 예측 비밀번호 길이의 범위
    """
    population = [] 
    for i in range(size): # size만큼의 염색체를 생성
        length = randint(min_len, max_len) # min_len ~ max_len에 해당하는 자연수를 임의로 추출
        population.append(generate_password(length)) # 생성된 염색체를 population에 추가
        
    return population

In [5]:
# 첫번째 세대 생성 -> min_len = 2, max_len = 10, size = 300
min_len = 2
max_len = 10
pop = generate_population(size=300, min_len=min_len, max_len=max_len) 
print(pop)

['CGYFdBV76', 'A6EMNL1GJr', 'VHYb5Q', '5wsx', 'ZW48EO', 'SlXzIbWuJo', 'BnlvuwCp', 'RD2ir8kUQ', '2yYSOr9R14', 'msH4Pf', 'N6J', '6EN', 'jnf', 'DMJWt4', 'PAOt', 'Iz4mCp', 'xiK', 'NAVToJC', 'fOqI', 'LF8fswIBy', 'B0JkQhdm', 'NxMIErs', 'ta', 'Y5EdUjcQ6', 'sxSp8vqtc2', 'jW', 'TGEoM0dW', 'OGz', 'eu', 'Uyk4d1qEY8', 'zl1kwn9I', 'YZlV', 'qfsT7x', 'mSZ', 'TXOs', 'r2laO', 'aPEydq', 'NY7SC6UOIi', 'R9', 'Zf4kpt', 'qdEI', 'DO', 'eukZli', 'Z6Jl', 'urqHKO7YwT', 'TmpcxeuS2', '6VQT79X', 'KCafDd49xo', 'pf', 'vjsHZ6', 'oI3CYb4', 'rTe0cRHzF', 'mMT1gAbp', 'F3B', '75nh9gct', 'i2', 'SzlFD3hW', 'OyB8Y2rwtA', 'zUEat1', 'FO3XVzK', 'V8MIn4cat2', 'd7', 'jTFYBL8o', 'o0w', 'tYw8', 'KvXBqND2R', 'mOAs8jBt', 'CIB', 'rx6a0yw', 'XF', 'L5Qf', 'B57', 'RK', 'nfd', 'zEkb0VS', 'W7pzcwG', '9ZEx', 'PX2traw', 'AP9Ofy4', 'BzpPbMWkEq', 'S25OuCdN6', 'hj9Lmt8Fi', 'b4RztQE', 'lLMJ1tS', '0sS', 'H9eFWO', '9r3zjq', 'LT75n4WDmK', 'Mbo67vVfs1', 'KMtx', 'JXA0', 'xFHVj6c2', 'ez3uoXU', 'sB5JofMK8Y', 'wrO', 'f4T5mX9VB', 'PZ8K0gk7b', 'qE1cI', 'P

### ● 적합도 함수(fitness evaluation)

적합도 함수는 각각의 염색체가 얼마나 정답과 일치하는지를 측정하는 함수입니다.  
본 튜토리얼인 비밀번호를 찾는 예제의 경우 염색체 각각의 비밀번호와의 일치 정도를 적합도 함수로 설정 할 수 있습니다.  
한편, 비밀번호와 염색체의 일치 정도의 기준은 1) 길이, 2) 문자열 일치 개수 2가지로 나눌 수 있습니다.  
1), 2)의 영향력을 얼마나 설정하는지에 따라 적합도가 바뀔 수 있으므로 본 튜토리얼에서는 1), 2)의 반영 비율을 하이퍼파라미터로 설정하여 다음과 같이 적합도 함수를 설정하고자 합니다.  

- 적합도 함수 산출 방식  
  
    - 길이(L)  
    염색체와 정답 비밀번호의 길이가 같은 경우 +1, 다른 경우 0  

    - 문자열 일치 점수(C)    
    case 1: 앞에서부터 문자열을 count 했을 때 문자열의 값과 위치가 일치하는 경우 -> +3
    case 2: 문자열의 위치는 다르지만 문자열의 값이 같은 경우 -> +1

길이(L), 문자열 일치 점수(C)와 하이퍼파라미터(alpha)를 통해 다음과 같은 적합도 함수를 산출  

적합도(fitness) = (1 + alpha * C)/max_score * 100


In [6]:
def fitness_function(password, chromosome, alpha):
    score = 0 # 초기 fitness 점수는 0으로 설정
    len_score = 0
    letter_score = 0

    if len(password) == len(chromosome):
        len_score = 1
    
    for i in range(len(password)):
        if password[i] in chromosome:
            letter_score+=1 # 문자열이 같은 값을 갖는 경우 +1
            
    for i in range(min(len(password),len(chromosome))):
        if password[i] == chromosome[i]:
            letter_score+=2 # 문자열이 같은 값을 갖는 경우에 점수인 1점은 위의 코드에서 반영
    
    score = len_score + alpha * letter_score
    fitness = score/(1+alpha*3*len(password)) * 100 # 100점 scale로 score 변환하여 fitness 값 산출

    return fitness


In [7]:
# fitness 함수 test(alpha 값에 따른 비교)
# alpha의 값이 커질수록 문자열의 일치 여부가 fitness에 기여하는 정도가 작아짐을 알 수 있음

alpha_list = [0.5*i for i in range(10)]

for alpha in alpha_list:
    print(f"Alpha: {alpha} | fitness: {fitness_function('12345', '34571',alpha)}")

Alpha: 0.0 | fitness: 100.0
Alpha: 0.5 | fitness: 35.294117647058826
Alpha: 1.0 | fitness: 31.25
Alpha: 1.5 | fitness: 29.78723404255319
Alpha: 2.0 | fitness: 29.03225806451613
Alpha: 2.5 | fitness: 28.57142857142857
Alpha: 3.0 | fitness: 28.26086956521739
Alpha: 3.5 | fitness: 28.037383177570092
Alpha: 4.0 | fitness: 27.86885245901639
Alpha: 4.5 | fitness: 27.73722627737226


### compute_performance 함수

각각의 chromosome과 password를 비교하여 fitness를 측정하고 정렬하는 함수

In [8]:
def compute_performance(population, password, alpha):
    performance_list = [] 
    performance_dict = {}
    for chromosome in population:
        fitness = fitness_function(password, chromosome, alpha)

        performance_list.append([chromosome, fitness])
        performance_dict[chromosome] = fitness

    # fitness 기준으로 내림차순 정령
    population_sorted_list = sorted(performance_list, key=lambda x: x[1], reverse=True)
    population_dict = performance_dict
    return population_sorted_list, population_dict

### Select 함수

각각의 chromosome 중 어떤 것을 선택할지 결정하는 select 함수에 대한 코드입니다.  
selection의 방법은 크게 deterministic method, probabilistic method로 나눌 수 있습니다.  
deterministic method의 경우 chromosome 중 fitness 값이 높은 상위 N개를 선택하는 방법이며,  
probabilistic method의 경우 fitness 값을 가중치로 주어 fitness 값이 높은 chromosome이 선택 될 확률을 높게 설정하는 방법론입니다.  
본 튜토리얼에서는 사용자가 deterministic, probablistic을 정하여 실험을 진행할 수 있도록 코드를 작성하였습니다.


In [9]:
def select_survivors(population_sorted_list, population_dict, num_best_sample, method):
    next_generation = []

    if method == 'deterministic':
        """
        상위 n개(num_best_sample)만큼 next_generation으로 번식
        """
        for i in range(num_best_sample):
            next_generation.append(population_sorted_list[i][0])
    
    if method == 'probablistic':
        """
        fitness값을 가중치로 하여 n개(num_best_sample)의 sample을 추출
        """
    
        next_generation_array = np.random.choice([key for key in population_dict], num_best_sample, p=[i/sum(population_dict.values()) for i in population_dict.values()])
        next_generation = next_generation_array.tolist()

    random.shuffle(next_generation)
    return next_generation

In [27]:
# 임의의 password 출력 실습

password = generate_password(length=10)
print("password is ", password)
min_len = 2
max_len = 10
pop = generate_population(size=100, min_len=min_len, max_len=max_len) 
pop_sorted, population_dict = compute_performance(pop, password, 5)

password is  flzPX8tImr


### Crossover & Mutation 함수

선택된 chromosome은 다음 세대의 부모가 됩니다. 선택된 chromosome을 통해 reproduction을 진행합니다.  
Reproduction 과정에서 crossover(교차)와 mutation(돌연변이)이 진행됩니다.  
Crossover의 경우 부모 세대의 유전자를 각각 50% 확률로 가져오는 것을 의미합니다.  
Mutation의 경우 local optimal에 빠지지 않도록 임의의 값으로 물려 받은 gene을 대체 해주는 작업을 의미합니다.  
Mutation의 정도를 mutation rate로 말하며 정답은 없지만 보통 0.01정도가 잘 작동하는 것으로 알려져있습니다.  
본 튜토리얼에서는 mutation rate의 기본값을 0.01로 설정하고 진행하겠습니다. 

In [15]:
# Crossover, chidren 생성 함수

def crossover(mother, father):
    child = ''
    # 코드의 편의를 위해 길이가 짧은 부모를 기준으로 유전자를 생성
    min_len = min(len(mother), len(father))

    for i in range(min_len):
        if (int(random.random())<50):
            child += mother[i] # 50%의 확률로 어머니의 유전자를 받음
        else:
            child += father[i] # 나머지 50%의 확률로 아버지의 유전자를 받음
    
    return child

def create_children(parents, n_child):
    next_population = []
    for i in range(int(len(parents)/2)):
        for j in range(n_child):
            next_population.append(crossover(parents[i], parents[len(parents) -1 -i]))
    return next_population

In [16]:
# Mutation
def mutate_word(word):
    # 비밀번호 길이 안에 포함되는 임의의 index 추출
    idx = int (random.random() * len(word)) 
    
    # 추출한 index에 random한 값 삽입
    if (idx == 0):
        word = random.choice(string.ascii_letters + string.digits) + word[1:]
    else:
        word = word[:idx] + random.choice(string.ascii_letters + string.digits) + word[idx+1:]
    return word

def mutate_population(population, chance_of_mutation):
    for i in range(len(population)):
        if random.random() * 100 < chance_of_mutation:
            population[i] = mutate_word(population[i])
    return population

## 실험 진행

In [97]:
if __name__ == "__main__":

    # 비밀번호 길이 범위는 2~10
    password = '56hiQqM2RZ'
    min_len = 2
    max_len = 10
    alpha_list = [2*i for i in range(1,6)]
    method_list = ["deterministic", "probablistic"]
    n_generation = 10000
    population = 100
    best_sample = 20
    n_child = 5
    chance_of_mutation_list = [2*i for i in range(1,6)]
    dict_alpha = {}
    for i in range(1,6):
        dict_alpha[f"alpha: {2*i}"] = 0
    
    deterministic_result = pd.DataFrame(dict_alpha, index = [f"mutation_rate: {2*i}" for i in range(1,6)])
    probablistic_result = pd.DataFrame(dict_alpha, index = [f"mutation_rate: {2*i}" for i in range(1,6)])

    # alpha = 2,4,6,8,10, 돌연변이 비율은 2,4,6,8,10, 총 300 세대를 생성하면서 반복한다.
    for method in method_list:
        for alpha in alpha_list:
            for chance_of_mutation in chance_of_mutation_list:
                pop = generate_population(size=population, min_len=min_len, max_len=max_len)
                for g in range(n_generation):
                    sucess_generation = n_generation
                    pop_sorted, population_dict = compute_performance(population=pop, password=password, alpha=alpha)

                    # pop_sorted에서 첫 번째의 점수가 100점이면 비밀번호 찾음.  
                    if int(pop_sorted[0][1]) == 100:
                        sucess_generation = g
                        break
                
                    survivors = select_survivors(population_sorted_list=pop_sorted, population_dict = population_dict, num_best_sample = best_sample, method= method)

                    children = create_children(parents=survivors, n_child=n_child)

                    new_generation = mutate_population(population=children, chance_of_mutation=chance_of_mutation)

                    pop = new_generation
                    
                if method == "deterministic":
                    if sucess_generation == n_generation:
                        deterministic_result[f'alpha: {alpha}'][f'mutation_rate: {chance_of_mutation}'] = 'fail'
                    else:
                        deterministic_result[f'alpha: {alpha}'][f'mutation_rate: {chance_of_mutation}'] = sucess_generation
                    
                elif method == "probablistic":
                    if sucess_generation == n_generation:
                        probablistic_result[f'alpha: {alpha}'][f'mutation_rate: {chance_of_mutation}'] = 'fail'
                    else:
                        probablistic_result[f'alpha: {alpha}'][f'mutation_rate: {chance_of_mutation}'] = sucess_generation
                    
                if sucess_generation == n_generation:
                    print(f"method: {method}, alpha :{alpha}, chance_of_mutation: {chance_of_mutation}은 비밀번호를 10000세대 안에 찾지 못하였습니다.")
                
                else:
                    print(f"method: {method}, alpha :{alpha}, chance_of_mutation: {chance_of_mutation}은 비밀번호를 {sucess_generation}세대에서 찾았습니다.")
    

method: deterministic, alpha :2, chance_of_mutation: 2은 비밀번호를 10000세대 안에 찾지 못하였습니다.


/home/seonggye/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


method: deterministic, alpha :2, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :2, chance_of_mutation: 6은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :2, chance_of_mutation: 8은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :2, chance_of_mutation: 10은 비밀번호를 10000세대 안에 찾지 못하였습니다.


/home/seonggye/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


method: deterministic, alpha :4, chance_of_mutation: 2은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :4, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :4, chance_of_mutation: 6은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :4, chance_of_mutation: 8은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :4, chance_of_mutation: 10은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :6, chance_of_mutation: 2은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :6, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :6, chance_of_mutation: 6은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :6, chance_of_mutation: 8은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :6, chance_of_mutation: 10은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :8, chance_of_mutation: 2은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :8, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾

/home/seonggye/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


method: probablistic, alpha :2, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :2, chance_of_mutation: 6은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :2, chance_of_mutation: 8은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :2, chance_of_mutation: 10은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :4, chance_of_mutation: 2은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :4, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :4, chance_of_mutation: 6은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :4, chance_of_mutation: 8은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :4, chance_of_mutation: 10은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :6, chance_of_mutation: 2은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :6, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :6, chance_of_mutation: 6은 비밀번호를 10000세대 안에 찾지 못하였습니다.
me

In [98]:
print(f"{'-'*20}deterministic_result{'-'*20}")
print(deterministic_result)
print(f"\n{'-'*20}probablistic_result{'-'*20}")
print(probablistic_result)

--------------------deterministic_result--------------------
                  alpha: 2 alpha: 4 alpha: 6 alpha: 8 alpha: 10
mutation_rate: 2      fail     fail     fail     fail      fail
mutation_rate: 4      fail     fail     fail     fail      fail
mutation_rate: 6      fail     fail     fail     fail      fail
mutation_rate: 8      fail     fail     fail     fail      fail
mutation_rate: 10     fail     fail     fail     fail      fail

--------------------probablistic_result--------------------
                  alpha: 2 alpha: 4 alpha: 6 alpha: 8 alpha: 10
mutation_rate: 2      fail     fail     fail     fail      fail
mutation_rate: 4      fail     fail     fail     fail      fail
mutation_rate: 6      fail     fail     fail     fail      fail
mutation_rate: 8      fail     fail     fail     fail      fail
mutation_rate: 10     fail     fail     fail     fail      fail


## 실험 결과 분석

위의 실험을 통해 method 조합, alpha, chance of mutation에 관계 없이 모두 비밀번호를 10000세대 안에 찾지 못한 것을 확인하였습니다.  
이는 적합도 설정에 문제가 있는 것으로 예상됩니다.  
비밀번호 해결의 난이도를 조금 더 쉽게 조정하기 위해 적합도 함수를 아래와 같이 수정하고 다시 유전 알고리즘을 진행하려고 합니다.  

- 적합도 수정  

    - 길이(L)  
    염색체와 정답 비밀번호의 길이가 같은 경우 +1점, 다른 경우는 문자열 일치 점수에 관계 없이 0점을 출력

    - 문자열 일치 점수(C)    
    case 1: 문자열의 값과 위치가 일치하는 경우 -> +3
    case 2: 문자열의 위치는 다르지만 문자열의 값이 같은 경우 -> +1

길이, 문자열 일치 점수(C)와 하이퍼파라미터(alpha)를 통해 다음과 같은 적합도를 산출  

1) 길이가 다른 경우: 0

2) 길이가 같은 경우 : (1 + alpha * C)/max_score * 100

즉 길이에 대한 페널티를 더 강하게 주어서 길이를 더 쉽게 찾을 수 있도록 적합도를 수정하였습니다.


In [28]:
def revised_fitness_function(password, chromosome, alpha):
    score = 0 # 초기 fitness 점수는 0으로 설정
    len_score = 0
    letter_score = 0

    if len(password) != len(chromosome):
        return score
    
    else:
        len_score = 1
    
    for i in range(len(password)):
        if password[i] in chromosome:
            letter_score+=1 # 문자열이 같은 값을 갖는 경우 +1
            
    for i in range(len(password)):
        if password[i] == chromosome[i]:
            letter_score+=2 # 문자열이 같은 값을 갖는 경우에 점수인 1점은 위의 코드에서 반영
    
    score = len_score + alpha * letter_score
    fitness = score/(1+alpha*3*len(password)) * 100 # 100점 scale로 score 변환하여 fitness 값 산출

    return fitness

In [30]:
def revised_compute_performance(population, password, alpha):
    performance_list = [] 
    performance_dict = {}
    for chromosome in population:
        fitness = revised_fitness_function(password, chromosome, alpha)

        performance_list.append([chromosome, fitness])
        performance_dict[chromosome] = fitness

    # fitness 기준으로 내림차순 정령
    population_sorted_list = sorted(performance_list, key=lambda x: x[1], reverse=True)
    population_dict = performance_dict
    return population_sorted_list, population_dict

## 수정한 적합도 함수를 이용한 실험

In [92]:
if __name__ == "__main__":

    # 비밀번호 길이 범위는 2~10
    password = '56hiQqM2RZ'
    min_len = 2
    max_len = 10
    alpha_list = [2*i for i in range(1,6)]
    method_list = ["deterministic", "probablistic"]
    n_generation = 10000
    population = 100
    best_sample = 20
    n_child = 5
    chance_of_mutation_list = [2*i for i in range(1,6)]
    dict_alpha = {}
    for i in range(1,6):
        dict_alpha[f"alpha: {2*i}"] = 0
    
    deterministic_result = pd.DataFrame(dict_alpha, index = [f"mutation_rate: {2*i}" for i in range(1,6)])
    probablistic_result = pd.DataFrame(dict_alpha, index = [f"mutation_rate: {2*i}" for i in range(1,6)])

    # alpha = 2,4,6,8,10, 돌연변이 비율은 2,4,6,8,10, 총 300 세대를 생성하면서 반복한다.
    for method in method_list:
        for alpha in alpha_list:
            for chance_of_mutation in chance_of_mutation_list:
                pop = generate_population(size=population, min_len=min_len, max_len=max_len)
                for g in range(n_generation):
                    sucess_generation = n_generation
                    pop_sorted, population_dict = revised_compute_performance(population=pop, password=password, alpha=alpha)

                    # pop_sorted에서 첫 번째의 점수가 100점이면 비밀번호 찾음.  
                    if int(pop_sorted[0][1]) == 100:
                        sucess_generation = g
                        break
                
                    survivors = select_survivors(population_sorted_list=pop_sorted, population_dict = population_dict, num_best_sample = best_sample, method= method)

                    children = create_children(parents=survivors, n_child=n_child)

                    new_generation = mutate_population(population=children, chance_of_mutation=chance_of_mutation)

                    pop = new_generation
                    
                if method == "deterministic":
                    if sucess_generation == n_generation:
                        deterministic_result[f'alpha: {alpha}'][f'mutation_rate: {chance_of_mutation}'] = 'fail'
                    else:
                        deterministic_result[f'alpha: {alpha}'][f'mutation_rate: {chance_of_mutation}'] = sucess_generation
                    
                elif method == "probablistic":
                    if sucess_generation == n_generation:
                        probablistic_result[f'alpha: {alpha}'][f'mutation_rate: {chance_of_mutation}'] = 'fail'
                    else:
                        probablistic_result[f'alpha: {alpha}'][f'mutation_rate: {chance_of_mutation}'] = sucess_generation
                    
                if sucess_generation == n_generation:
                    print(f"method: {method}, alpha :{alpha}, chance_of_mutation: {chance_of_mutation}은 비밀번호를 10000세대 안에 찾지 못하였습니다.")
                
                else:
                    print(f"method: {method}, alpha :{alpha}, chance_of_mutation: {chance_of_mutation}은 비밀번호를 {sucess_generation}세대에서 찾았습니다.")
    

method: deterministic, alpha :2, chance_of_mutation: 2은 비밀번호를 1928세대에서 찾았습니다.
method: deterministic, alpha :2, chance_of_mutation: 4은 비밀번호를 894세대에서 찾았습니다.
method: deterministic, alpha :2, chance_of_mutation: 6은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :2, chance_of_mutation: 8은 비밀번호를 663세대에서 찾았습니다.


/home/seonggye/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/seonggye/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


method: deterministic, alpha :2, chance_of_mutation: 10은 비밀번호를 10000세대 안에 찾지 못하였습니다.


/home/seonggye/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


method: deterministic, alpha :4, chance_of_mutation: 2은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :4, chance_of_mutation: 4은 비밀번호를 1168세대에서 찾았습니다.
method: deterministic, alpha :4, chance_of_mutation: 6은 비밀번호를 812세대에서 찾았습니다.
method: deterministic, alpha :4, chance_of_mutation: 8은 비밀번호를 392세대에서 찾았습니다.
method: deterministic, alpha :4, chance_of_mutation: 10은 비밀번호를 457세대에서 찾았습니다.
method: deterministic, alpha :6, chance_of_mutation: 2은 비밀번호를 1942세대에서 찾았습니다.
method: deterministic, alpha :6, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :6, chance_of_mutation: 6은 비밀번호를 631세대에서 찾았습니다.
method: deterministic, alpha :6, chance_of_mutation: 8은 비밀번호를 575세대에서 찾았습니다.
method: deterministic, alpha :6, chance_of_mutation: 10은 비밀번호를 467세대에서 찾았습니다.
method: deterministic, alpha :8, chance_of_mutation: 2은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: deterministic, alpha :8, chance_of_mutation: 4은 비밀번호를 1709세대에서 찾았습니다.
method: deterministic, alpha :8, chance_of_mutatio

/home/seonggye/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


method: probablistic, alpha :2, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :2, chance_of_mutation: 6은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :2, chance_of_mutation: 8은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :2, chance_of_mutation: 10은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :4, chance_of_mutation: 2은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :4, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :4, chance_of_mutation: 6은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :4, chance_of_mutation: 8은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :4, chance_of_mutation: 10은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :6, chance_of_mutation: 2은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :6, chance_of_mutation: 4은 비밀번호를 10000세대 안에 찾지 못하였습니다.
method: probablistic, alpha :6, chance_of_mutation: 6은 비밀번호를 10000세대 안에 찾지 못하였습니다.
me

In [96]:
print(f"{'-'*20}deterministic_result{'-'*20}")
print(deterministic_result)
print(f"\n{'-'*20}probablistic_result{'-'*20}")
print(probablistic_result)

--------------------deterministic_result--------------------
                  alpha: 2 alpha: 4 alpha: 6 alpha: 8 alpha: 10
mutation_rate: 2      1928     fail     1942     fail      fail
mutation_rate: 4       894     1168     fail     1709      1837
mutation_rate: 6      fail      812      631      954      fail
mutation_rate: 8       663      392      575     fail       609
mutation_rate: 10     fail      457      467      879       580

--------------------probablistic_result--------------------
                  alpha: 2 alpha: 4 alpha: 6 alpha: 8 alpha: 10
mutation_rate: 2      fail     fail     fail     fail      fail
mutation_rate: 4      fail     fail     fail     fail      fail
mutation_rate: 6      fail     fail     fail     fail      fail
mutation_rate: 8      fail     fail     fail     fail      fail
mutation_rate: 10     fail     fail     fail     fail      fail


## 실험결과 분석  
본 실험에서는 probablistic method는 잘 작동하지 않는 것을 확인하였습니다.  
적합도 계산 방식을 수정하여 실험을 진행하였고, deterministic하게 유전자를 선택하는 경우에는 어느 정도 잘 작동하는 것을 확인하였습니다.  
가장 빠른 세대에서 비밀번호를 찾았던 경우는 alpha = 4, mutation_rate = 10 인 경우였습니다.  
일반적으로 mutation_rate가 1% 정도인 경우가 잘 작동하는 것으로 알려져 있으나, 본 예제에서는 세대수가 많지 않고 비교적 단순한 작업이었기 때문에 이와 같이 작동한 것으로 예상됩니다.  
추가적인 하이퍼파라미터 탐색과 더 쉬운 난이도의 적합도 계산 방식을 통해 더 빠르게 비밀번호를 탐색 할 수 있을 것이라 예상됩니다.
